In [1]:
# go to source directory for this project
%cd A:\Users\skjal\Code\Master\in5550\final\in5550-final 

A:\Users\skjal\Code\Master\in5550\final\in5550-final


In [2]:
import random
import argparse
import torch
import torch.nn.functional as F
from torch import nn
from torchtext import data, vocab
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import f1_score
from helpers import *
from Models import *

A:\Users\skjal\Miniconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#def main():
    #torch.manual_seed(1337)
    #random.seed(13370)

    #parser = argparse.ArgumentParser()
    #parser.add_argument('--train', action='store')
    #parser.add_argument('--dev', action='store')
    #parser.add_argument('--embeds', action='store')
    #args = parser.parse_args()
    
args = argparse.Namespace()
args.train = "nli5550/nli5550.train.jsonl"
args.dev = "nli5550/nli5550.dev.jsonl"
args.test = "nli5550/nli5550.test.jsonl"
args.embeds = "../model.txt"

In [4]:

token_field = data.Field(sequential=True, batch_first=True, include_lengths=True, tokenize=lambda x: x.split(), preprocessing=lambda x: x[1].split())
label_field = data.Field(sequential=False, batch_first=True, preprocessing=lambda x: x[1])
NaF = ('none', None)

fields = [
            NaF, #('annotator_labels', label_field), 
            NaF, # captionID
            ('gold_label', label_field), 
            NaF, # pairID
            NaF, #('sentence1', token_field),
            NaF, #('sentence1_parse', token_field),
            NaF, #('sentence2', token_field),
            NaF, #('sentence2_parse', token_field),
            ('sentence1_tok', token_field),
            ('sentence2_tok', token_field)
        ]


In [5]:
split_train_data = False
if split_train_data:
    train_dataset = data.Dataset( load_jsonl_examples(args.train, fields) , fields=fields).split(0.2)[0]
else:
    train_dataset = data.Dataset( load_jsonl_examples(args.train, fields) , fields=fields)
dev_dataset = data.Dataset( load_jsonl_examples(args.dev, fields) , fields=fields)
test_dataset = data.Dataset( load_jsonl_examples(args.test, fields) , fields=fields)

In [6]:
print( 'train_dataset lenght:\t{}'.format(len(train_dataset)))
print( 'dev_dataset lenght:\t{}'.format(len(dev_dataset)))
print( 'test_dataset lenght:\t{}'.format(len(test_dataset)))

train_dataset lenght:	550152
dev_dataset lenght:	10000
test_dataset lenght:	10000


In [7]:
# Step 3: build vocabs
#_vecs = vocab.Vectors(args.embeds)
_vecs = "glove.6B.100d"
use_test = True
if use_test:
    token_field.build_vocab(test_dataset, vectors=_vecs)
    label_field.build_vocab(test_dataset)
    train_iter = data.Iterator(test_dataset, batch_size=32, train=True, sort=True, repeat=False, sort_key=lambda x: (len(x.sentence1_tok)+len(x.sentence1_tok))/2)
else:
    token_field.build_vocab(train_dataset, vectors=_vecs)
    label_field.build_vocab(train_dataset)
    train_iter = data.Iterator(train_dataset, batch_size=32, train=True, sort=True, repeat=False, sort_key=lambda x: (len(x.sentence1_tok)+len(x.sentence1_tok))/2)

dev_iter = data.Iterator(dev_dataset, batch_size=1, train=False, sort=False)


In [8]:
class noteboookmodel2(nn.Module):
    def __init__(self, token_vocab, tag_vocab):
        super().__init__()
        
        input_size = token_vocab.vectors.shape[1]*16 # output size of the encoder
        hidden_state_size = 512
        #self._embeds = nn.Embedding.from_pretrained(token_vocab.vectors)
        self._embeds = nn.Embedding(len(token_vocab), 300, padding_idx=token_vocab.stoi['<pad>']).from_pretrained(token_vocab.vectors)
        # define the rest of your components
        self._conv1 = nn.Conv1d(in_channels=token_vocab.vectors.shape[1],
                                out_channels=token_vocab.vectors.shape[1],
                                kernel_size=3, 
                                padding=1)
        self._conv2 = nn.Conv1d(in_channels=token_vocab.vectors.shape[1],
                                out_channels=token_vocab.vectors.shape[1],
                                kernel_size=3, 
                                padding=1)
        self._conv3 = nn.Conv1d(in_channels=token_vocab.vectors.shape[1],
                                out_channels=token_vocab.vectors.shape[1],
                                kernel_size=3, 
                                padding=1)
        self._conv4 = nn.Conv1d(in_channels=token_vocab.vectors.shape[1],
                                out_channels=token_vocab.vectors.shape[1],
                                kernel_size=3, 
                                padding=1)

        self.MLP = nn.Sequential(nn.Linear(input_size, hidden_state_size), # input layer
                                 nn.ReLU(), # hidden layer
                                 nn.Linear(hidden_state_size, len(tag_vocab))) # ouput layer
    def forward(self, batch):
        verbose = False
        #tokens, lengths = batch
        tokens = [batch.sentence1_tok, batch.sentence2_tok]
        
        for i, token in enumerate(tokens):
            token, lengths = token
            
            # embeddings go here
            embeds = self._embeds(token).transpose(1, 2)
            if verbose: print("embeds",embeds.shape)
            # conv + pooling
            c1 = self._conv1(embeds)
            if verbose: print("c1",c1.shape)
            c2 = self._conv2(c1)
            if verbose: print("c2",c2.shape)
            c3 = self._conv3(c2)
            if verbose: print("c3",c3.shape)
            c4 = self._conv4(c2)
            if verbose: print("c4",c4.shape)
            #tokens[i] = F.dropout(torch.cat([c1.max(dim=-1)[0], c2.max(dim=-1)[0], c3.max(dim=-1)[0], c4.max(dim=-1)[0]], dim=1), p=0.33, training=self.training)
            tokens[i] = torch.cat([c1.max(dim=-1)[0], c2.max(dim=-1)[0], c3.max(dim=-1)[0], c4.max(dim=-1)[0]], dim=1)
            if verbose: print("sentence {}".format(i), tokens[i].shape)
        
        u, v = tokens
        
        # [u,v |u-v|, u*v]
        a = torch.cat([u, v], dim=1)
        b = torch.abs(u-v)
        c = u*v
        vec = torch.cat([a, b, c], dim=1)
        if verbose: print('vec', vec.shape)
        return self.MLP(vec)

In [9]:




def train(model, iterator, criterion, optimiser, epoch, writer=None):
    
    model.train()
    #encoder.train()
    #pbar = tqdm(total=len(iterator.dataset))
    pbar = tqdm(total=len(iterator.data()))
    accuracy, total = 0, 0
    total_loss = 0
    iterator.init_epoch()
    for n, batch in enumerate(iterator):
        optimiser.zero_grad()
        #u = encoder(batch.sentence1_tok)
        #v = encoder(batch.sentence2_tok)

        predictions = model(batch)
        gold = batch.gold_label
        
        loss = criterion(predictions, gold)
        
        predmax = predictions.argmax(dim=-1)
        total_loss += loss.item()
        accuracy += ( predmax== gold).nonzero().size(0)
        total += predmax.size(0)
        
        loss.backward()
        optimiser.step()
        pbar.update(32) # batch_size
        pbar.set_postfix(loss=loss.item())
        if writer is not None: writer.add_scalar('Train/Loss', loss, n)
        
    if writer is not None: writer.add_scalar('Train/total_loss', total_loss, epoch)
    if writer is not None: writer.add_scalar('Train/mean_loss', total_loss/total, epoch)
    if writer is not None: writer.add_scalar('Train/accuracy', accuracy/total, epoch)
    
    pbar.close()

def evaluate(model, iterator, criterion, epoch, writer=None):
    model.eval()
    accuracy, total = 0, 0
    total_loss = 0
    for n, batch in enumerate(iterator):
        prediction = model(batch)
        predictions = prediction.argmax(dim=-1)
        gold = batch.gold_label
        loss = criterion(prediction, gold)
        total_loss += loss
        accuracy += (predictions == gold).nonzero().size(0)
        total += predictions.size(0)
        if writer is not None: writer.add_scalar('Dev/Loss', loss, n)
    
    if writer is not None: writer.add_scalar('Dev/total_loss', total_loss, epoch)
    if writer is not None: writer.add_scalar('Dev/mean_loss', total_loss/total, epoch)
    if writer is not None: writer.add_scalar('Dev/accuracy', accuracy/total, epoch)
    print("> dev accuracy: {}/{} = {}".format(accuracy, total, accuracy/total))

In [10]:
import math
import torch
import torch.nn.functional as F

from torch import nn

    

    
    
class self_attention_Encoder(nn.Module):
    def __init__(self, token_vocab, hidden_size=1024, dimension_a=100, attention_hops=4):
        super().__init__()
        self.hidden_size = hidden_size
        
        # word embeddings
        #self.embedding = nn.Embedding.from_pretrained(token_vocab.vectors)
        self.embedding = nn.Embedding(len(token_vocab), 300, padding_idx=token_vocab.stoi['<pad>']).from_pretrained(token_vocab.vectors)

        # to achive bi-direction i stack 2 LSTMCells, one for each direction; RNN_1 (->), RNN_2 (<-)
        self.RNN_1 = nn.LSTMCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        self.RNN_2 = nn.LSTMCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        
        #self.RNN_1 = nn.RNNCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        #self.RNN_2 = nn.RNNCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        
        
        # alpha weights
        Ws1 = torch.zeros(dimension_a, hidden_size*2)
        nn.init.normal_(Ws1, mean=0.0, std=1.0)
        self.Ws1 = nn.Parameter(Ws1)
        
        Ws2 = torch.zeros(attention_hops, dimension_a)
        nn.init.normal_(Ws2, mean=1.0, std=1.0)
        self.Ws2 = nn.Parameter(Ws2)
        
    def forward(self, sentence):
        tokens, _ = sentence;
        batch_size = len(_)
        H_1 = [] # layer 1, ->
        H_2 = [] # layer 2, <-
        A = [] # softmax(ws2*tanh(ws1*Ht))
        
        # get embedding vectors
        embedded = self.embedding(tokens)
        embedded = embedded.transpose(1,0)
     
        
        # initial word for layer 1
        hidden_state = self.RNN_1(embedded[0])
        H_1.append(hidden_state)
        
        # layer 1 RNN
        for token in embedded[1:]:
            hidden_state = self.RNN_1(token, hidden_state)
            H_1.append(hidden_state)
        
        # flip the words to do layer 2
        embedded = torch.flip(embedded, [0])
        
        # initial word for layer 2
        hidden_state = self.RNN_2(embedded[0])
        H_2.append(hidden_state)
       
        # layer 2 RNN
        for token in embedded[1:]:
            hidden_state = self.RNN_2(token, hidden_state)
            H_2.append(hidden_state)
       
        #flip H_2 so the hidden states line up with H_1
        H_2.reverse()
            
        # concat each hidden states for H_1, H_2
        H = []
        for h1, h2 in zip(H_1, H_2):
            H.append(torch.cat([h1[0], h2[0]], dim=1))
        H = torch.stack(H) # turn python list of tensors into 1 tensor for the hidden states
        H = H.transpose(1,0)
        
        # for every sentence in batch calculate attention
        for batched_H in H:
            A.append(F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ batched_H.transpose(0,1)), dim=1) )
        
        
        # attention distrubution
        A = torch.stack(A)
        u = A@H # weighted sums
        u = u.view((batch_size, -1, 1))
        u = torch.squeeze(u, -1)
        return u


In [12]:
class self_attention_Encoder3(nn.Module):
    def __init__(self, token_vocab, hidden_size=1024, dimension_a=100, attention_hops=3):
        super().__init__()
        self.hidden_size = hidden_size
        
        # word embeddings
        #self.embedding = nn.Embedding.from_pretrained(token_vocab.vectors)
        self.embedding = nn.Embedding(len(token_vocab), 300, padding_idx=token_vocab.stoi['<pad>']).from_pretrained(token_vocab.vectors)

        # to achive bi-direction i stack 2 LSTMCells, one for each direction; RNN_1 (->), RNN_2 (<-)
        self.RNN_1 = nn.LSTMCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        self.RNN_2 = nn.LSTMCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        
        #self.RNN_1 = nn.RNNCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        #self.RNN_2 = nn.RNNCell(token_vocab.vectors.shape[1], hidden_size, bias=True)
        
        
        # alpha weights
        Ws1 = torch.zeros(dimension_a, hidden_size*2)
        nn.init.normal_(Ws1, mean=0.0, std=1.0)
        self.Ws1 = nn.Parameter(Ws1)
        
        Ws2 = torch.zeros(attention_hops, dimension_a)
        nn.init.normal_(Ws2, mean=1.0, std=1.0)
        self.Ws2 = nn.Parameter(Ws2)
        
    def forward(self, sentence):
        tokens, _ = sentence;
        batch_size = len(_)
        H_1 = [] # layer 1, ->
        H_2 = [] # layer 2, <-
        A = [] # softmax(ws2*tanh(ws1*Ht))
        
        # get embedding vectors
        embedded = self.embedding(tokens)
        embedded = embedded.transpose(1,0)
     
        
        # initial word for layer 1
        hidden_state = self.RNN_1(embedded[0])
        H_1.append(hidden_state)
        
        # layer 1 RNN
        for token in embedded[1:]:
            hidden_state = self.RNN_1(token, hidden_state)
            H_1.append(hidden_state)
        
        # flip the words to do layer 2
        embedded = torch.flip(embedded, [0])
        
        # initial word for layer 2
        hidden_state = self.RNN_2(embedded[0])
        H_2.append(hidden_state)
       
        # layer 2 RNN
        for token in embedded[1:]:
            hidden_state = self.RNN_2(token, hidden_state)
            H_2.append(hidden_state)
       
        #flip H_2 so the hidden states line up with H_1
        H_2.reverse()
        
        # concat each hidden states for H_1, H_2
        H = []
        for h1, h2 in zip(H_1, H_2):
            H.append(torch.cat([h1[0], h2[0]], dim=1))

        H = torch.stack(H) # turn python list of tensors into 1 tensor for the hidden states
        H = H.transpose(1,0)
        
        a = [ F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ h.t()), dim=1) for h in H ]
        
        # attention distrubution
        A = torch.stack(a)
        #go through all in batch and do operation
        v = []
        for a, h in zip(A,H):
            v.append(a@h)
        u = torch.stack(v)
        #print('u',u.shape)
        u_ = [ torch.cat([v for v in u_], dim=-1) for u_ in u ]
        u = torch.stack(u_)
       
        #print('U',u.shape)
        # batch_size x attention_hops x hidden_size*2
        return u


In [82]:
class MLP(nn.Module):
    def __init__(self, token_vocab, tag_vocab, encoder, input_size=1024, hidden_size=512):
        super().__init__()
        self.encoder = encoder
        input_size = input_size * 2 * 4 * 4
        self.MLP = nn.Sequential(nn.Linear(input_size, hidden_size), # input layer
                                 nn.ReLU(), # hidden layer
                                 nn.Linear(hidden_size, len(tag_vocab))) # ouput layer
    def forward(self, batch):
        u = self.encoder(batch.sentence1_tok)
        v = self.encoder(batch.sentence2_tok)
        
        # [u, v |u-v|, u*v]
        vec = torch.cat([torch.cat([u, v], dim=1), torch.abs(u-v), u*v], dim=1)
        
        return self.MLP(vec)

In [116]:
class self_attention_Encoder2(nn.Module):
    def __init__(self, token_vocab, hidden_size=1024, dimension_a=256, attention_hops=4):
        super().__init__()
        self.hidden_size = hidden_size
        
        # word embeddings
        #self.embedding = nn.Embedding.from_pretrained(token_vocab.vectors)
        self.embedding = nn.Embedding(len(token_vocab), 300, padding_idx=token_vocab.stoi['<pad>']).from_pretrained(token_vocab.vectors)

        self.RNN = nn.LSTM(token_vocab.vectors.shape[1], hidden_size, bias=True, batch_first=True, bidirectional=True)
        #self.RNN_2 = nn.LSTM(token_vocab.vectors.shape[1], hidden_size, bias=True, batch_first=True, bidirectional=True)
        
        self.MLP = nn.Sequential(nn.Linear(hidden_size*2, dimension_a), # input layer
                                 nn.Tanh(), # hidden layer
                                 nn.Linear(dimension_a, attention_hops), # ouput layer
                                 nn.Softmax(dim=1)
                                )
        # alpha weights
        #Ws1 = torch.zeros(dimension_a, hidden_size*2)
        #nn.init.normal_(Ws1, mean=0.0, std=1.0)
        #self.Ws1 = nn.Parameter(Ws1)
        
        #Ws2 = torch.zeros(attention_hops, dimension_a)
        #nn.init.normal_(Ws2, mean=1.0, std=1.0)
        #self.Ws2 = nn.Parameter(Ws2)
        
    def forward(self, sentence):
        tokens, lengths = sentence;
        #print('len', lengths[0])
        lengths = list(torch.unbind(lengths))
        tokens = list(torch.unbind(tokens))
        A = [] 
        
        batch = []
        for i, (t, l) in enumerate(zip(tokens, lengths)):
            batch.append( [i, t, l, None] )
            
        batch = sorted(batch, reverse=True, key=lambda x: x[2]) # sort by lengths
        
        tokens = torch.stack([x[1] for x in batch])
        lengths = torch.stack([x[2] for x in batch])
        
        # get embedding vectors
        embedded = self.embedding(tokens)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True)
        
        out, (hidden, _) = self.RNN(packed)
        out, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        #hidden = torch.cat([h for h in hidden], dim=1)
        #print('hidden', hidden.data.shape)
        #for h in out.transpose(0,1):
            #A.append(F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ h.t()), dim=1))
        #view(num_layers, num_directions, batch, hidden_size)
        #print('hidden',hidden.shape)
        #print('out',out.shape)   
        #a = [ F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ h.t()), dim=1) for h in out ]
        # attention distrubution
        #A = torch.stack(a)
        #print('out',out.transpose(0,1).shape, 'A', A.shape)
        A = self.MLP(out).transpose(2,1)
        u = A@out
        
        # add the vectors to the batch list
        for i, v in enumerate(u):
            batch[i][3] = v
            
        batch = sorted(batch, key=lambda x: x[0]) # sort by original batch indexes
        u = torch.stack([x[3] for x in batch])
        u = u.transpose(1,0)
        
        u = torch.cat([v for v in u], dim=1)
        #print('rep u', u.shape)
        return u

In [ ]:
#net = noteboookmodel2(token_field.vocab, label_field.vocab)
#enc = Baseline(token_field.vocab)
enc = self_attention_Encoder2(token_field.vocab)
net = MLP(token_field.vocab, label_field.vocab, enc)
print(sum(p.numel() for p in net.parameters() if p.requires_grad))

criterion = nn.CrossEntropyLoss()
#optimiser = torch.optim.Adam([{'params': net.parameters()}, {'params': enc.parameters()}], lr=1e-3)
optimiser = torch.optim.Adam(net.parameters(), lr=1e-3)

from tensorboardX import SummaryWriter

#from torch.utils.tensorboard import SummaryWriter
# Writer will output to ./runs/ directory by default
MODEL_NAME = "mlp-attention_1_run2"
writer = SummaryWriter('./runs/{}'.format(MODEL_NAME))

for epoch in range(5):
    train(net, train_iter, criterion, optimiser, epoch, writer)
    evaluate(net, dev_iter, criterion, epoch, writer)
        
    


26530057


In [ ]:
class self_attention_Encoder2_2(nn.Module):
    def __init__(self, token_vocab, hidden_size=1024, dimension_a=256, attention_hops=4):
        super().__init__()
        self.hidden_size = hidden_size
        
        # word embeddings
        #self.embedding = nn.Embedding.from_pretrained(token_vocab.vectors)
        self.embedding = nn.Embedding(len(token_vocab), 300, padding_idx=token_vocab.stoi['<pad>']).from_pretrained(token_vocab.vectors)

        self.RNN = nn.LSTM(token_vocab.vectors.shape[1], hidden_size, bias=True, batch_first=True, bidirectional=True)
        #self.RNN_2 = nn.LSTM(token_vocab.vectors.shape[1], hidden_size, bias=True, batch_first=True, bidirectional=True)
        
        self.MLP = nn.Sequential(nn.Linear(hidden_size*2, dimension_a), # input layer
                                 nn.Tanh(), # hidden layer
                                 nn.Linear(dimension_a, attention_hops), # ouput layer
                                 nn.Softmax(dim=1)
                                )
        # alpha weights
        #Ws1 = torch.zeros(dimension_a, hidden_size*2)
        #nn.init.normal_(Ws1, mean=0.0, std=1.0)
        #self.Ws1 = nn.Parameter(Ws1)
        
        #Ws2 = torch.zeros(attention_hops, dimension_a)
        #nn.init.normal_(Ws2, mean=1.0, std=1.0)
        #self.Ws2 = nn.Parameter(Ws2)
        
    def forward(self, sentence):
        tokens, lengths = sentence;
        #print('len', lengths[0])
        lengths = list(torch.unbind(lengths))
        tokens = list(torch.unbind(tokens))
        A = [] 
        
        batch = []
        for i, (t, l) in enumerate(zip(tokens, lengths)):
            batch.append( [i, t, l, None] )
            
        batch = sorted(batch, reverse=True, key=lambda x: x[2]) # sort by lengths
        
        tokens = torch.stack([x[1] for x in batch])
        lengths = torch.stack([x[2] for x in batch])
        
        # get embedding vectors
        embedded = self.embedding(tokens)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True)
        
        out, (hidden, _) = self.RNN(packed)
        out, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        #hidden = torch.cat([h for h in hidden], dim=1)
        #print('hidden', hidden.data.shape)
        #for h in out.transpose(0,1):
            #A.append(F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ h.t()), dim=1))
        #view(num_layers, num_directions, batch, hidden_size)
        #print('hidden',hidden.shape)
        #print('out',out.shape)   
        #a = [ F.softmax(self.Ws2 @ F.tanh(self.Ws1 @ h.t()), dim=1) for h in out ]
        # attention distrubution
        #A = torch.stack(a)
        #print('out',out.transpose(0,1).shape, 'A', A.shape)
        A = self.MLP(out).transpose(2,1)
        u = A@out
        u = torch.sum(u, 1)
        # add the vectors to the batch list
        for i, v in enumerate(u):
            batch[i][3] = v
            
        batch = sorted(batch, key=lambda x: x[0]) # sort by original batch indexes
        u = torch.stack([x[3] for x in batch])
        u = u.transpose(1,0)
        
        u = torch.cat([v for v in u], dim=1)
        #print('rep u', u.shape)
        return u

In [ ]:
#net = noteboookmodel2(token_field.vocab, label_field.vocab)
#enc = Baseline(token_field.vocab)
enc = self_attention_Encoder2_2(token_field.vocab)
net = MLP(token_field.vocab, label_field.vocab, enc)
print(sum(p.numel() for p in net.parameters() if p.requires_grad))

criterion = nn.CrossEntropyLoss()
#optimiser = torch.optim.Adam([{'params': net.parameters()}, {'params': enc.parameters()}], lr=1e-3)
optimiser = torch.optim.Adam(net.parameters(), lr=1e-3)

from tensorboardX import SummaryWriter

#from torch.utils.tensorboard import SummaryWriter
# Writer will output to ./runs/ directory by default
MODEL_NAME = "mlp-attention_1_max"
writer = SummaryWriter('./runs/{}'.format(MODEL_NAME))

for epoch in range(5):
    train(net, train_iter, criterion, optimiser, epoch, writer)
    evaluate(net, dev_iter, criterion, epoch, writer)
        
    


In [ ]:
#tensorboard.notebook.start('--logdir ./runs')
#http://www.erogol.com/use-tensorboard-pytorch/#viewSource